In [ ]:
!pip install -qq scikit-image captum Pillow pydicom torchinfo torchview

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.1 MB/s eta 0:00:00


In [ ]:
import os
import random
import re
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import requests

# Google
from google.colab import drive
drive.mount('/content/drive/')

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Pydicom - Load DCM Files
import pydicom as dicom
from pydicom.pixel_data_handlers.util import apply_modality_lut, apply_voi_lut

# Scikit-image
from skimage.io import imread

# Pytorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import torch.nn.functional as F
from torchvision.models import resnet18, resnet50
from torchinfo import summary
from torchview import draw_graph

# Captum
import captum
from captum.attr import Saliency, LayerGradCam, Occlusion
from captum.attr import visualization as viz

Mounted at /content/drive/


In [ ]:
# Load the Google Sheet using Pandas.
sheet_id = "1iVB7gvQdXeJ_N0vvYxWYmQvWT_Nwnb8CFsdH-rV6YGs"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

,patient_id,path,new_path,label_id,label_name,herniated_disc,no_stenosis,facet_joint,foramen,thecal_sac
0,1,/content/drive/MyDrive/thesis/Datasets/lumbar_...,/1/,/Labeller 1/,/Herniated Disc/,1.0,2.0,20.0,13.0,12.0
1,2,/content/drive/MyDrive/thesis/Datasets/lumbar_...,/2/,/Labeller 2/,/No Stenosis/,3.0,9.0,22.0,15.0,14.0
2,3,/content/drive/MyDrive/thesis/Datasets/lumbar_...,/3/,/Labeller 3/,/Thecal Sac/,4.0,10.0,26.0,17.0,16.0
3,4,/content/drive/MyDrive/thesis/Datasets/lumbar_...,/4/,/Labeller 4/,/Foramen/,5.0,18.0,30.0,19.0,23.0
4,5,/content/drive/MyDrive/thesis/Datasets/lumbar_...,/5/,/Labeller 5/,/Facet Joint/,6.0,24.0,52.0,21.0,25.0


In [ ]:
# Define the labels.

def get_label(selected_label_name: str):
  label_name = list(df['label_name'])

  for i in range(0, len(label_name)):
    if '/Herniated Disc/' in selected_label_name:
      return 'Herniated Disc'
    if '/No Stenosis/' in selected_label_name:
      return 'No Stenosis'
    if '/Thecal Sac/' in selected_label_name:
      return 'Thecal Sac'
    if '/Foramen/' in selected_label_name:
      return 'Foramen'
    if '/Facet Joint/' in selected_label_name:
      return 'Facet Joint'

print(get_label('/Herniated Disc/'))

Herniated Disc


In [ ]:
# load all the MRIs on the basis of patient_ids.

training_dataset_path = '/content/drive/MyDrive/thesis/Datasets/lumbar_spinal_cord_mri_datasets/preprocessed_images/final_preprocessed/training'
testing_dataset_path = '/content/drive/MyDrive/thesis/Datasets/lumbar_spinal_cord_mri_datasets/preprocessed_images/final_preprocessed/testing'
paths = list(df['new_path'])

def load_dataset(dataset_path: str):
  mri_dcm = []
  mri_dcm.clear()

  dataset_list = list(Path(dataset_path).rglob("*.jpg" ))
  for fileName in dataset_list:
    try:
      file_size = os.path.getsize(fileName)
      if file_size > 0:
        # for i in paths:
        # if i in str(fileName):
            # patient_id = str(re.findall('\d+',i)[0])
            # new_data = {'filePath': str(fileName), 'patient_id': patient_id}
        new_data = {'filePath': str(fileName), 'label': get_label(str(fileName))}
        mri_dcm.append(new_data)
    except FileNotFoundError as e:
      continue
  return mri_dcm

training_dataset = load_dataset(training_dataset_path)
testing_dataset = load_dataset(testing_dataset_path)

In [ ]:
training_dataset[0]

{'filePath': '/content/drive/MyDrive/thesis/Datasets/lumbar_spinal_cord_mri_datasets/preprocessed_images/final_preprocessed/training/Herniated Disc/image_2.jpg',
 'label': 'Herniated Disc'}

Show the total number of training dataset available

In [ ]:
len(training_dataset), len(testing_dataset)

(1455, 624)

Creating own Dataset `Class` to load our own datasets.


In [ ]:
def get_label_no(selected_label_name: str):
  if selected_label_name == 'Herniated Disc':
    return 1
  if selected_label_name == 'Thecal Sac':
    return 2
  if selected_label_name == 'Foramen':
    return 3
  if selected_label_name == 'Facet Joint':
    return 4
  if selected_label_name == 'No Stenosis':
    return 5

In [ ]:
class LumbarSpinalDataset(Dataset):
  def __init__(self, dataset = None, transform=None, image_size = None):
    self.labels = dataset
    self.transform = transform
    self.image_size = image_size

    self.classes = ['Herniated Disc', 'No Stenosis', 'Thecal Sac', 'Foramen', 'Facet Joint']
    self.load_dataset()

  def load_dataset(self):
    image_path = []

    for i in range(0, len(training_dataset)):
      image_path.append((training_dataset[i]['filePath'], get_label_no(training_dataset[i]['label'])))

    for i in range(0, len(testing_dataset)):
      image_path.append((testing_dataset[i]['filePath'], get_label_no(testing_dataset[i]['label'])))

    self.labels = image_path

  def normalize(self, image):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
    image = transform(image)
    return image

  def __getitem__(self, index):
    fpath, target = self.labels[index]
    # fpath = self.labels[idx]
    image_array = imread(fpath, as_gray=True)
    # image_array = self.normalize_image(image_array)
    image_array = self.normalize(image_array)
    # data = torch.from_numpy(image_array)
    data = image_array.type(torch.FloatTensor)
    # data = torch.unsqueeze(data, 0)
    data = transforms.Resize((self.image_size, self.image_size))(data)

    return data, target

  def __len__(self):
    return len(self.labels)

Call our `Dataset` custom class.

In [ ]:
dataset = LumbarSpinalDataset(image_size=256)

In [ ]:
type(dataset)

__main__.LumbarSpinalDataset

### Spliting the `Dataset` Randomly

For this we are splitting our dataset into training, validation, and testing by 70%, 10% and 20% respectively.

In [ ]:
train_fraction = 0.7
val_fraction = 0.1
test_fraction = 0.2
dataset_size = len(dataset)
print(dataset_size)

num_train = int(train_fraction * dataset_size)
num_validation = int(val_fraction * dataset_size)
num_test = dataset_size - num_train - num_validation

print(num_train, num_validation, num_test)

train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(
                                 dataset, [num_train, num_validation, num_test],
                                 generator=torch.Generator().manual_seed(42))
print(type(train_dataset))
print(len(train_dataset), len(validation_dataset), len(test_dataset))

2079
1455 207 417
<class 'torch.utils.data.dataset.Subset'>
1455 207 417


Check do we conneted with the GPU or not.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Dataloaders

It is hard for us to train our model by giving it one by one images in each epoch. So, this procedure is quite slow, for this we need to make sure that in each epoch we will pass a batch of images instead of a single image. That is why, we are using `Dataloaders` to load our data in `batches`.

In [ ]:
train_batch_size = 200
eval_batch_size = 10

train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=eval_batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=eval_batch_size)

In [ ]:
len(train_dataloader), len(validation_dataloader), len(test_dataloader)

(8, 21, 42)

We need to make sure that in every systems the random values must be same.

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

Build a Model

In [ ]:
# resnetX = (Num of channels, repetition, Bottleneck_expansion , Bottleneck_layer)
model_parameters={}
model_parameters['resnet18'] = ([64,128,256,512],[2,2,2,2],1,False)
model_parameters['resnet34'] = ([64,128,256,512],[3,4,6,3],1,False)
model_parameters['resnet50'] = ([64,128,256,512],[3,4,6,3],4,True)
model_parameters['resnet101'] = ([64,128,256,512],[3,4,23,3],4,True)
model_parameters['resnet152'] = ([64,128,256,512],[3,8,36,3],4,True)

In [ ]:
class Bottleneck(torch.nn.Module):

    def __init__(self,in_channels,intermediate_channels,expansion,is_Bottleneck,stride):

        """
        Creates a Bottleneck with conv 1x1->3x3->1x1 layers.

        Note:
          1. Addition of feature maps occur at just before the final ReLU with the input feature maps
          2. if input size is different from output, select projected mapping or else identity mapping.
          3. if is_Bottleneck=False (3x3->3x3) are used else (1x1->3x3->1x1). Bottleneck is required for resnet-50/101/152
        Args:
            in_channels (int) : input channels to the Bottleneck
            intermediate_channels (int) : number of channels to 3x3 conv
            expansion (int) : factor by which the input #channels are increased
            stride (int) : stride applied in the 3x3 conv. 2 for first Bottleneck of the block and 1 for remaining

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """

        super(Bottleneck,self).__init__()

        self.expansion = expansion
        self.in_channels = in_channels
        self.intermediate_channels = intermediate_channels
        self.is_Bottleneck = is_Bottleneck

        # i.e. if dim(x) == dim(F) => Identity function
        if self.in_channels==self.intermediate_channels*self.expansion:
            self.identity = True
        else:
            self.identity = False
            projection_layer = []
            projection_layer.append(nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels*self.expansion, kernel_size=1, stride=stride, padding=0, bias=False ))
            projection_layer.append(nn.BatchNorm2d(self.intermediate_channels*self.expansion))
            # Only conv->BN and no ReLU
            # projection_layer.append(nn.ReLU())
            self.projection = nn.Sequential(*projection_layer)

        # commonly used relu
        self.relu = nn.ReLU()

        # is_Bottleneck = True for all ResNet 50+
        if self.is_Bottleneck:
            # bottleneck
            # 1x1
            self.conv1_1x1 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False )
            self.batchnorm1 = nn.BatchNorm2d(self.intermediate_channels)

            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False )
            self.batchnorm2 = nn.BatchNorm2d(self.intermediate_channels)

            # 1x1
            self.conv3_1x1 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels*self.expansion, kernel_size=1, stride=1, padding=0, bias=False )
            self.batchnorm3 = nn.BatchNorm2d( self.intermediate_channels*self.expansion )

        else:
            # basicblock
            # 3x3
            self.conv1_3x3 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False )
            self.batchnorm1 = nn.BatchNorm2d(self.intermediate_channels)

            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=1, padding=1, bias=False )
            self.batchnorm2 = nn.BatchNorm2d(self.intermediate_channels)

    def forward(self,x):
        # input stored to be added before the final relu
        in_x = x

        if self.is_Bottleneck:
            # conv1x1->BN->relu
            x = self.relu(self.batchnorm1(self.conv1_1x1(x)))

            # conv3x3->BN->relu
            x = self.relu(self.batchnorm2(self.conv2_3x3(x)))

            # conv1x1->BN
            x = self.batchnorm3(self.conv3_1x1(x))

        else:
            # conv3x3->BN->relu
            x = self.relu(self.batchnorm1(self.conv1_3x3(x)))

            # conv3x3->BN
            x = self.batchnorm2(self.conv2_3x3(x))


        # identity or projected mapping
        if self.identity:
            x += in_x
        else:
            x += self.projection(in_x)

        # final relu
        x = self.relu(x)

        return x


# Bottleneck(64*4,64,4,stride=1)

def test_Bottleneck():
    x = torch.randn(1,64,112,112)
    model = Bottleneck(64,64,4,True,2)
    print(model(x).shape)
    del model

test_Bottleneck()

torch.Size([1, 256, 56, 56])


In [ ]:
class ResNet(nn.Module):

    def __init__(self, resnet_variant,in_channels,num_classes):
        """
        Creates the ResNet architecture based on the provided variant. 18/34/50/101 etc.
        Based on the input parameters, define the channels list, repeatition list along with expansion factor(4) and stride(3/1)
        using _make_blocks method, create a sequence of multiple Bottlenecks
        Average Pool at the end before the FC layer

        Args:
            resnet_variant (list) : eg. [[64,128,256,512],[3,4,6,3],4,True]
            in_channels (int) : image channels (3)
            num_classes (int) : output #classes

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """
        super(ResNet,self).__init__()
        self.channels_list = resnet_variant[0]
        self.repeatition_list = resnet_variant[1]
        self.expansion = resnet_variant[2]
        self.is_Bottleneck = resnet_variant[3]

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False )
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.block1 = self._make_blocks( 64 , self.channels_list[0], self.repeatition_list[0], self.expansion, self.is_Bottleneck, stride=1 )
        self.block2 = self._make_blocks( self.channels_list[0]*self.expansion , self.channels_list[1], self.repeatition_list[1], self.expansion, self.is_Bottleneck, stride=2 )
        self.block3 = self._make_blocks( self.channels_list[1]*self.expansion , self.channels_list[2], self.repeatition_list[2], self.expansion, self.is_Bottleneck, stride=2 )
        self.block4 = self._make_blocks( self.channels_list[2]*self.expansion , self.channels_list[3], self.repeatition_list[3], self.expansion, self.is_Bottleneck, stride=2 )

        self.average_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear( self.channels_list[3]*self.expansion , num_classes)



    def forward(self,x):
        x = self.relu(self.batchnorm1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = self.block4(x)

        x = self.average_pool(x)

        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)

        return x

    def _make_blocks(self,in_channels,intermediate_channels,num_repeat, expansion, is_Bottleneck, stride):

        """
        Args:
            in_channels : #channels of the Bottleneck input
            intermediate_channels : #channels of the 3x3 in the Bottleneck
            num_repeat : #Bottlenecks in the block
            expansion : factor by which intermediate_channels are multiplied to create the output channels
            is_Bottleneck : status if Bottleneck in required
            stride : stride to be used in the first Bottleneck conv 3x3

        Attributes:
            Sequence of Bottleneck layers

        """
        layers = []

        layers.append(Bottleneck(in_channels,intermediate_channels,expansion,is_Bottleneck,stride=stride))
        for num in range(1,num_repeat):
            layers.append(Bottleneck(intermediate_channels*expansion,intermediate_channels,expansion,is_Bottleneck,stride=1))

        return nn.Sequential(*layers)


def test_ResNet(params):
    model = ResNet( params , in_channels=3, num_classes=1000)
    x = torch.randn(1,3,224,224)
    output = model(x)
    print(output.shape)
    return model

architecture = 'resnet50'
model = test_ResNet(model_parameters[architecture])

torch.Size([1, 1000])


In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
from torchvision.models import resnet34,resnet50,resnet18,resnet101,resnet152,ResNet50_Weights
torchvision_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

from torchsummary import summary
summary(torchvision_model, (3, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 44.7MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
# # net = resnet18()
# net = resnet50()
# net.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# net = net.to(device)

In [ ]:
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
# epochs = 6

Train a model and pass training and validation data to it.

In [ ]:
best_validation_accuracy = 0
best_testing_accuracy = 0
# batch_index, inputs, targets = None, None, None

# For training visualization later
train_accs = []
val_accs = []
test_accs = []

# training loop
for epoch in range(epochs):
  # Set a network to train mode, so that its parameters can be changed
  net.train()

  # print training info
  print("Epoch {}:".format(epoch))

  # Statistics needed to compute classification accuracy.
  # the total number of image examples trained on
  total_train_examples = 0

  # the number of examples classified correctly
  num_correct_train = 0

  # iterate over the training set once
  # for batch_index, (inputs) in tqdm(enumerate(train_dataloader), total=len(train_dataset) // train_batch_size):
  # for batch_index, inputs, targets in tqdm(enumerate(train_dataloader), total=len(train_dataset) // train_batch_size, desc="Training Process"):
  for batch_index, (inputs, targets) in tqdm(enumerate(train_dataloader), total=len(train_dataset) // train_batch_size, desc="Processing Data"):
    inputs = inputs.to(device)
    targets = targets.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    predictation = net(inputs)
    loss = criterion(predictation, targets)
    loss.backward()
    optimizer.step()

    _, predicted_class = predictation.max(1)
    total_train_examples += predicted_class.size(0)
    num_correct_train += predicted_class.eq(targets).sum().item()

  train_acc = num_correct_train / total_train_examples
  print("Training accuracy: {}".format(train_acc))
  train_accs.append(train_acc)

  total_val_examples = 0
  num_correct_val = 0

  net.eval()

  with torch.no_grad():
    for batch_index, (inputs, targets) in tqdm(enumerate(validation_dataloader), total=len(validation_dataset) // eval_batch_size, desc="validation"):
      inputs = inputs.to(device)
      targets = targets.to(device)

      predictation = net(inputs)
      _, predicted_class = predictation.max(1)
      total_val_examples += predicted_class.size(0)
      num_correct_val += predicted_class.eq(targets).sum().item()

  val_acc = num_correct_val / total_val_examples
  print("Validation accuracy: {}".format(val_acc))
  val_accs.append(val_acc)

  total_test_examples = 0
  num_correct_test = 0

  with torch.no_grad():
    for batch_index, (inputs, targets) in tqdm(enumerate(test_dataloader), total=len(test_dataset) // eval_batch_size, desc="testing"):
      inputs = inputs.to(device)
      targets = targets.to(device)

      predictation = net(inputs)
      _, predicted_class = predictation.max(1)
      total_test_examples += predicted_class.size(0)
      num_correct_test += predicted_class.eq(targets).sum().item()

  test_acc = num_correct_test / total_test_examples
  print("Testing accuracy: {}".format(test_acc))
  test_accs.append(test_acc)

  if val_acc > best_validation_accuracy:
    best_validation_accuracy = val_acc
    # torch.save(net.state_dict(), "best_model.pth")
    print("validation accuracy improved, saving model")
    net_copy = deepcopy(net)

  if test_acc > best_testing_accuracy:
    best_testing_accuracy = test_acc
    # torch.save(net.state_dict(), "best_model.pth")
    print("testing accuracy improved, saving model")
    net_copy = deepcopy(net)